In [16]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import xgboost as xgb
import numpy
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from numpy import argmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import random
%matplotlib inline

inPath = './datas/first_train_data_20180131/'


In [17]:
train_index = pd.read_csv('./datas/first_train_index_20180131.csv')

In [18]:
test_index = pd.read_csv('./datas/first_test_index_20180131.csv')

In [19]:
train_index.head()
test_index.head()

,id
0,822405
1,1142964
2,1241977
3,1309049
4,1085810


In [20]:
def parseFeatureByRow(row, hasLabel=True):
    id = str(row.id)
    if hasLabel:
        info = pd.read_csv('./datas/first_train_data_20180131/' + id + '.txt',header=None).loc[0];
    else:
        info = pd.read_csv('./datas/first_test_data_20180131/' + id + '.txt',header=None).loc[0];
    if hasLabel:
        info[len(info)] = row.y1
        info[len(info)] = row.y2
        info[len(info)] = row.y3
        info[len(info)] = row.y4
    return info;

In [48]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(20, input_dim=2600, activation='relu'))
	model.add(Dense(4, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [49]:
# sample = train_index.sample(1000, replace=True)
# train_d = [parseFeatureByRow(row) for idx, row in sample.iterrows()]
# train_d = pd.DataFrame(train_d)
# train_d.head()

In [50]:
encoder = LabelEncoder()
encoder.fit(train_index.iloc[:,-1])
encoded_Y = encoder.transform(train_index.iloc[:,-1])
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

array([[ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.]])

In [51]:
tran_index_label = pd.concat([train_index, pd.DataFrame(dummy_y,columns=['y1', 'y2', 'y3', 'y4'])], axis=1)

In [52]:
def genearte_batch_data(index_data, batch_size=5, hasLabel= True):
    while (True):
        y1 = index_data.loc[index_data.y1==1,:].sample(batch_size,replace=True)
        y2 = index_data.loc[index_data.y2==1,:].sample(batch_size,replace=True)
        y3 = index_data.loc[index_data.y3==1,:].sample(batch_size,replace=True)
        y4 = index_data.loc[index_data.y4==1,:].sample(batch_size,replace=True)
        batch = pd.concat([y1, y2, y3, y4])
#         return (batch)
        train_d = [parseFeatureByRow(row,hasLabel) for idx, row in batch.iterrows()]
        train_d = pd.DataFrame(train_d)
        x = train_d.iloc[:,:-4]
        y = train_d.iloc[:,-4:]
#         return x
        yield x, y


In [53]:
def genearte_test_batch_data(index_data, batch_size=5):
    i = 0
    while (True):
        batch = index_data.loc[(i*batch_size):(i+1)*batch_size]
        i=i+1
        train_d = [parseFeatureByRow(row,hasLabel=False) for idx, row in batch.iterrows()]
        train_d = pd.DataFrame(train_d)
        x = train_d
        print('finished:'+i)
        yield x

In [54]:
genearte_batch_data(tran_index_label)

<generator object genearte_batch_data at 0x1a224e98c0>

In [55]:
batch_size = 100
epochs = 1
model = baseline_model()
bst = model.fit_generator(genearte_batch_data(tran_index_label, batch_size), steps_per_epoch=1, epochs=epochs, workers=10)


Epoch 1/1
1/1 [==============================] - 92s 92s/step - loss: 10.0918 - acc: 0.2775


In [303]:
steps = test_index.size/batch_size;
pred = model.predict_generator(genearte_test_batch_data(test_index, batch_size), steps=steps, workers=4)


In [301]:
batch_size

20

In [304]:
def submit(preds, test_feature, fileName):
    submission = test_feature[['id']]
    final_pred = encoder.inverse_transform(argmax(preds,axis=1))
    submission['pred'] = pd.Series(final_pred)
    submission.to_csv('./predicts/'+fileName, header=False, index=False)
    return submission

In [306]:
rs  =submit(pred, test_index, '0214_2_dnn.csv')

array(['star', 'galaxy', 'unknown', 'qso'], dtype=object)

array(['star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star'], dtype=object)

In [158]:
genearte_batch_data(train_index,5)

<generator object genearte_batch_data at 0x1a55d43e10>

In [106]:
X = train_d.iloc[:,:-1]
seed = 7
numpy.random.seed(seed)
estimator = KerasClassifier(build_fn=baseline_model, epochs=1000,  verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))





Baseline: 48.10% (42.55%)


array([ 0.02,  0.03,  0.08,  0.9 ,  0.9 ,  0.93,  0.91,  0.08,  0.89,  0.07])